# Linux/Python Tutorial
#### Lesson 3: Using Flask

This lesson will closely follow [this tutorial](https://www.raspberrypi.org/learning/python-web-server-with-flask/worksheet/), but will put it in a more useable IPython Notebook format.

## Setup

In [ ]:
!sudo apt-get install python-pip

In [ ]:
!sudo pip install flask

## Navigating to the right place

In [ ]:
%%bash
cd ~
mkdir -p webapp

## Flask Code

This cell will create a text file called `app.py` in the current directory, which is `webapp`. Since this text file ends in `.py`, we can run it as a Python program. We write the code to run the Flask server in the cell below. That way, if we want to change something, we can just run the following cell again and the code for the server will be overwritten with the new code.

In [ ]:
%%writefile ~/webapp/app.py

from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return 'Katies Cool ' * 1000

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')

To run the server, run the following cell. The bash cell magic with the `--bg` flags will allow you to keep using the IPython notebook while the server is run in the *background*. 

In [ ]:
%%bash --bg
cd ~/webapp/
python app.py

To access from some other device on the same network, navigate to:


In [ ]:
host = !hostname -I
'http://' + host[0][:-1] + ':5000/'

## Templates

You can use HTML templates to make more formatted web pages. We'll first create a directory to store these templates called `templates`.

In [ ]:
%%bash
cd ~/webapp
mkdir -p templates

Let's write a simple template. This cell will write the HTML code into and HTML file in `templates`.

In [ ]:
%%writefile ~/webapp/templates/index.html

<html>
<body>
<h1>Hello from a template!</h1>
</body>
</html>

Then rewrite the Flask application to call the template

In [ ]:
%%writefile ~/webapp/app.py

from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')

## Shutting the Server Down

Keeping the server running will keep port 5000 occupied at all times. This isn't really a big deal, but in case you want to shut the server down we can write in a route that will shut down the Flask server if we navigate to it. Notice the code below adds the route `/shutdown` and kills the server. Run the following, then try to navigate to `127.0.0.1/shutdown`.

In [ ]:
%%writefile ~/webapp/app.py

from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

from flask import request

def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

@app.route('/shutdown', methods=['GET', 'POST'])
def shutdown():
    shutdown_server()
    return 'Server shutting down...'

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')

To get the server running again, run the following:

In [ ]:
%%bash --bg
cd ~/webapp/
python app.py

Let's use a link within the index template to shutdown the server instead of remembering where to navigate each time we want to shut down.

In [ ]:
%%writefile ~/webapp/templates/index.html

<html>
<body>
<h1>Hello, World!</h1>
<a href="/shutdown">Shutdown Server</a>
</body>
</html>

Refresh your page, then try the shutdown link. To get the server running again, run the following

In [ ]:
%%bash --bg
cd ~/webapp/
python app.py

Now that you've got the basics, you can try [this](http://mattrichardson.com/Raspberry-Pi-Flask/index.html) physical computing project.

## Sharing your Flask server with a friend

First, we'll have to install npm and localtunnel, then make a shortcut so that things work properly. This might take a while...

In [ ]:
%%bash
sudo apt-get update
sudo apt-get install npm
sudo npm install -g localtunnel
sudo ln -s /usr/bin/nodejs /usr/bin/node

Then run this cell. It will occupy the IPython notebook while it serves your Flask application to the outside world. If you'd rather it run in the background, just use the bash cell magic with the --bg flag.

In [ ]:
!lt --port 5000

You won't be able to access the page at 127.0.0.1:5000 anymore, but you can access it from the URL above now. When you shutdown the localtunnel command above, you'll also kill the Flask server. Every time you run the command above, you'll get a different URL.